In [12]:
import os
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np

In [4]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [8]:
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [7]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model = model.to(device)

In [18]:
MODEL_EPOCH = 0

models_folder = "trained_models"

model_path = os.path.join(models_folder, f"gpt2_medium_joker_{MODEL_EPOCH}.pt")
model.load_state_dict(torch.load(model_path))

jokes_output_file_path = f'generated_{MODEL_EPOCH}.jokes'

model.eval()
if os.path.exists(jokes_output_file_path):
    os.remove(jokes_output_file_path)
    
joke_num = 0
with torch.no_grad():
   
        for joke_idx in range(1000):
        
            joke_finished = False

            cur_ids = torch.tensor(tokenizer.encode("JOKE:")).unsqueeze(0).to(device)

            for i in range(100):
                outputs = model(cur_ids, labels=cur_ids)
                loss, logits = outputs[:2]
                softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(from only one in this case) batch and the last predicted embedding
                if i < 3:
                    n = 20
                else:
                    n = 3
                next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n) #Randomly(from the topN probability distribution) select the next word
                cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word to the running sequence

                if next_token_id in tokenizer.encode('<|endoftext|>'):
                    joke_finished = True
                    break

            
            if joke_finished:
                
                joke_num = joke_num + 1
                
                output_list = list(cur_ids.squeeze().to('cpu'))
                # print(output_list)
                output_text = tokenizer.decode(output_list)

                with open(jokes_output_file_path, 'a') as f:
                    f.write(f"{output_text} \n\n")

[tensor(45006), tensor(7336), tensor(25), tensor(40), tensor(550), tensor(284), tensor(1011), tensor(616), tensor(3290), tensor(329), tensor(281), tensor(30278), tensor(13), tensor(679), tensor(373), tensor(257), tensor(1310), tensor(1643), tensor(15175), tensor(13), tensor(50256)]
[tensor(45006), tensor(7336), tensor(25), tensor(2215), tensor(314), tensor(717), tensor(2067), tensor(10691), tensor(257), tensor(2576), tensor(11), tensor(314), tensor(373), tensor(523), tensor(6568), tensor(284), tensor(1826), tensor(257), tensor(3516), tensor(508), tensor(373), tensor(588), tensor(11), tensor(366), tensor(10814), tensor(0), tensor(314), tensor(1053), tensor(1392), tensor(257), tensor(11077), tensor(2474), tensor(314), tensor(373), tensor(655), tensor(588), tensor(11), tensor(366), tensor(40), tensor(1101), tensor(407), tensor(1016), tensor(284), tensor(307), tensor(257), tensor(11077), tensor(329), tensor(428), tensor(3516), tensor(526), tensor(314), tensor(1101), tensor(407), tensor(257